# Find bars nearby soccer stadiums located in Germany

In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from geopedestrian.services import drive_from, solve_walking, walk_to
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory

In [2]:
class LocalClient(GeoRapidClient):
    
    def __init__(self, url, auth_headers):
        pass
    
    @property
    def auth_headers(self):
        return None
    
    @property
    def url(self):
        return 'http://localhost:7071/api'

## If in debug mode using localhost

In [4]:
client = LocalClient(None, None)

TypeError: __init__() missing 2 required positional arguments: 'url' and 'auth_headers'

## If using the cloud service being hosted on RapidAPI

In [6]:
host = 'geopedestrian.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

## Location of Olympiastadion, Berlin

In [19]:
lat, lon = 52.514610, 13.239665

In [20]:
walking_geojson = solve_walking(client, lat, lon)
walking_fset = FeatureSet.from_geojson(walking_geojson)
walking_fset.sdf

,ObjectID,FacilityID,Name,FromBreak,ToBreak,Shape_Length,Shape_Area,SHAPE
0,1,1,Location 1 : 10 - 15,10,15,0.15438372101428938,0.00016588132385536136,"{""rings"": [[[13.241466144000071, 52.5223607870..."
1,2,1,Location 1 : 5 - 10,5,10,0.09693572565674625,8.546242622503289e-05,"{""rings"": [[[13.241915299000027, 52.5194412760..."
2,3,1,Location 1 : 0 - 5,0,5,0.028781889442187052,2.7108911552395858e-05,"{""rings"": [[[13.241466144000071, 52.5165217650..."


In [21]:
gis = GIS()

In [22]:
berlin_map = gis.map('Berlin, Germany')

In [23]:
walking_fset.sdf.spatial.plot(berlin_map)
berlin_map

MapView(layout=Layout(height='400px', width='100%'))

## Places Category Bars

In [25]:
category = 13003
walking_result = walk_to(client, lat, lon, category)
places_geojson = walking_result['places']
places_fset = FeatureSet.from_geojson(places_geojson)
places_fset.sdf

,placeId,name,from,to,OBJECTID,SHAPE
0,5cde1c9e2ea35f16f014c2f138a3f67d,Jesse Owens Lounge,0,5,1,"{""x"": 13.239891, ""y"": 52.513868, ""spatialRefer..."
1,adc3424f4bb5b49b592375c80335f58c,Olympia-Lounge,0,5,2,"{""x"": 13.239891, ""y"": 52.513868, ""spatialRefer..."
2,f4b6b2028a398e02f8ce89504e6ec964,Players Lounge,0,5,3,"{""x"": 13.239693, ""y"": 52.514409, ""spatialRefer..."


In [13]:
places_fset.sdf.spatial.plot(berlin_map)
berlin_map

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)

In [14]:
routes_geojson = walking_result['routes']
routes_fset = FeatureSet.from_geojson(routes_geojson)
routes_fset.sdf

,name,total_minutes,total_kilometers,OBJECTID,SHAPE
0,Location 1 - Players Lounge,0.07,0.01,1,"{""paths"": [[[13.23969456900005, 52.51360921700..."
1,Location 1 - Jesse Owens Lounge,0.21,0.02,2,"{""paths"": [[[13.23969456900005, 52.51360921700..."
2,Location 1 - Olympia-Lounge,0.21,0.02,3,"{""paths"": [[[13.23969456900005, 52.51360921700..."


In [15]:
routes_fset.sdf.spatial.plot(berlin_map)
berlin_map

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)

## Places Ambulance Service

In [27]:
ambulance_category = 15009
near_lat, near_lon = 52.5062753, 13.3454184
driving_result = drive_from(client, near_lat, near_lon, ambulance_category)
places_geojson = driving_result['places']
places_fset = FeatureSet.from_geojson(places_geojson)
places_fset.sdf

,placeId,name,from,to,OBJECTID,SHAPE
0,0a9655f16f4cc02666df5af7a9bfc70f,Krankentransport ASB Regionalverband Nordost,0,5,1,"{""x"": 13.356262, ""y"": 52.498727, ""spatialRefer..."
1,cb35611a9f5632113cc5b5582db4370c,Blitz-Mobil Fahrdienst UG,5,15,2,"{""x"": 13.334868, ""y"": 52.4954, ""spatialReferen..."


In [17]:
routes_geojson = driving_result['routes']
routes_fset = FeatureSet.from_geojson(routes_geojson)
routes_fset.sdf

,name,total_minutes,total_kilometers,OBJECTID,SHAPE
0,Krankentransport ASB Regionalverband Nordost -...,5.11,1.8,1,"{""paths"": [[[13.356293522000044, 52.4988830670..."
1,Blitz-Mobil Fahrdienst UG - Location 1,5.51,1.59,2,"{""paths"": [[[13.334616023000024, 52.4953933130..."


In [18]:
routes_fset.sdf.spatial.plot(berlin_map)
berlin_map

MapView(jupyter_target='notebook', layout=Layout(height='400px', width='100%'), ready=True)